In [2]:
# from proteinbert import load_pretrained_model
# from proteinbert.conv_and_global_attention_model import get_model_with_hidden_layers_as_outputs
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
from tensorflow.keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

2023-08-29 17:33:51.969522: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9498] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-08-29 17:33:51.969565: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-08-29 17:33:51.971325: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-08-29 17:33:52.072474: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-29 17:33:52.899228: W tensorflow/compiler/

Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.


In [4]:
df = pd.read_csv('mutated.csv')

In [5]:
# Assuming 'df' is your DataFrame
protein_sequences = df['mutated_sequence']
delta_g_values = df['delta_G']

# Define the amino acids and their order
amino_acids = "ACDEFGHIKLMNPQRSTVWY"

def one_hot_encode_sequence(sequence):
    # Create a dictionary to map each amino acid to an index
    aa_to_index = {aa: i for i, aa in enumerate(amino_acids)}
    
    # Initialize an array of zeros with the shape (sequence_length, number_of_amino_acids)
    sequence_length = len(sequence)
    one_hot_encoded = np.zeros((sequence_length, len(amino_acids)), dtype=int)
    
    # Set the appropriate index to 1 for each amino acid in the sequence
    for i, aa in enumerate(sequence):
        one_hot_encoded[i, aa_to_index[aa]] = 1
    
    return one_hot_encoded

# One-hot encode all the protein sequences in the DataFrame
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Assuming you have a list of one-hot encoded sequences
encoded_sequences = [one_hot_encode_sequence(seq) for seq in protein_sequences]

#Find the maximum sequence length
max_sequence_length = max(len(seq) for seq in encoded_sequences)

# Pad sequences to the maximum length
padded_sequences = pad_sequences(encoded_sequences, maxlen=max_sequence_length, padding='post', truncating='post', dtype='float32')

# Convert the padded_sequences list to a numpy array
X = np.array(padded_sequences)

# Convert the delta_g_values to a numpy array
y = np.array(delta_g_values)
del protein_sequences
del delta_g_values
del encoded_sequences
del padded_sequences


In [6]:
# Split the data into 70% training and 30% test
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3,random_state=42)

# # Split the remaining data (30%) into 10% validation and 20% test
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.67, random_state=42)

# # Print the shapes of the datasets
print("Training data shapes:", X_train.shape, y_train.shape)
print("Validation data shapes:", X_val.shape, y_val.shape)
print("Test data shapes:", X_test.shape, y_test.shape)


Training data shapes: (1469, 34350, 20) (1469,)
Validation data shapes: (207, 34350, 20) (207,)
Test data shapes: (423, 34350, 20) (423,)


In [7]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau

In [ ]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor
X_train = np.reshape(X_train, (X_train.shape[0], -1))
y_train = np.reshape(y_train, (-1,))

# Initialize lists to store R2 scores for each fold
r2_scores = []
mse = []
# Create a KFold cross-validator
num_folds = 3
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

# Iterate through cross-validation folds
for train_idx, val_idx in kf.split(X_train):
    # Split the data into train and validation sets for this fold
    X_fold_train, X_fold_val = X_train[train_idx], X_train[val_idx]
    y_fold_train, y_fold_val = y_train[train_idx], y_train[val_idx]
    
    # Create a new instance of the random forest regressor for each fold
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    
    # Train the model on the current fold's training data
    model.fit(X_fold_train, y_fold_train)
    
    # Evaluate the model on the current fold's validation data
    y_fold_val_pred = model.predict(X_fold_val)
    r2 = r2_score(y_fold_val, y_fold_val_pred)
    mse_i = mean_squared_error(y_fold_val, y_fold_val_pred)
    mse.append(mse_i)
    r2_scores.append(r2)

# Calculate the mean and standard deviation of R2 scores
mean_r2 = np.mean(r2_scores)
std_r2 = np.std(r2_scores)
mse_mean = np.mean(mse)
print(f"Mean R2 across {num_folds} folds: {mean_r2:.4f}")
print(f"Standard Deviation of R2: {std_r2:.4f}")
print(f"Mean square error : {mse_mean:.4f}")
